# Fazendo otimização com naive bayes

Nesse notebook vamos tentar fazer otimização de hiperparametros com naive bayes e comparar os resultados sem feito a otimização

In [2]:
# importando os módulos
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV , RandomizedSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.naive_bayes import GaussianNB

In [3]:
heart = pd.read_csv('heart_failure_clinical_records_dataset.csv')

In [4]:
x = heart.drop(['DEATH_EVENT'],axis=1).values
y = heart.DEATH_EVENT.values

x_std = StandardScaler().fit_transform(x)

In [5]:
nm = RandomUnderSampler(random_state=10)

x_nm , y_nm = nm.fit_resample(x_std,y)

## Aplicando o Grid Search

In [29]:
parametros = {'var_smoothing':np.linspace(0, 10)}

In [30]:
grid_search = GridSearchCV(estimator=GaussianNB(),param_grid=parametros).fit(x_nm,y_nm)

In [31]:
melhores_parametros = grid_search.best_params_
melhor_resultado = grid_search.best_score_
print(melhores_parametros)
print(melhor_resultado)

{'var_smoothing': 0.0}
0.6920377867746288


In [25]:
na = GaussianNB(var_smoothing=0.02040816326530612)

nome_metricas = ['accuracy', 'precision_macro', 'recall_macro']

metricas_ran = cross_validate(na,x_nm, y_nm, cv=7, scoring=nome_metricas)
for met in metricas_ran:
  print(f'-{met}')
  print(f"-- {metricas_ran[met]}")
  media = np.mean(metricas_ran[met])
  desvio = np.std(metricas_ran[met])
  print(f'Média do {met}: {media}')
  print(f'Desvio {desvio}')
  print(f'Intervalo [{(media-(2*desvio)):.3f},{(media+(2*desvio)):.3f}]')
  print('-*-'*20)

-fit_time
-- [0.00233006 0.00084233 0.00075698 0.00082135 0.0007267  0.00072408
 0.00073552]
Média do fit_time: 0.0009910038539341518
Desvio 0.000548389118121634
Intervalo [-0.000,0.002]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-score_time
-- [0.0026381  0.00182962 0.0017724  0.0017426  0.00168109 0.00173736
 0.00168896]
Média do score_time: 0.0018700190952845982
Desvio 0.00031700017202064584
Intervalo [0.001,0.003]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_accuracy
-- [0.82142857 0.82142857 0.71428571 0.77777778 0.74074074 0.66666667
 0.44444444]
Média do test_accuracy: 0.7123960695389268
Desvio 0.12115349386477614
Intervalo [0.470,0.955]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_precision_macro
-- [0.8368984  0.82307692 0.81818182 0.80555556 0.83333333 0.72857143
 0.39583333]
Média do test_precision_macro: 0.7487786839677595
Desvio 0.14806078025828115
Intervalo [0.453,1.045]
-*--*--*--*--*--*--*--*--*--*--*--*--*--

Comparndo as metricas, temos:

- naive bayes *sem otimização*:
  - acuracia: 0,722
  - precisão: 0,754
  - recall: 0,724
- naive bayes *com grid search*:
  - acuracia: 0,712
  - precisão: 0,748
  - recall: 0,713


## Aplicando random search

In [32]:
random_search = RandomizedSearchCV(estimator=GaussianNB(),param_distributions=parametros)
random_search.fit(x_nm,y_nm)
melhores_parametros2 = random_search.best_params_
melhor_resultado2 = random_search.best_score_

In [33]:
print(melhores_parametros2)
print(melhor_resultado2)

{'var_smoothing': 1.0204081632653061}
0.6241565452091767


In [34]:
na2 = GaussianNB(var_smoothing=1.0204081632653061)

In [35]:

metricas_ran2 = cross_validate(na2,x_nm, y_nm, cv=7, scoring=nome_metricas)
for met in metricas_ran:
  print(f'-{met}')
  print(f"-- {metricas_ran2[met]}")
  media = np.mean(metricas_ran2[met])
  desvio = np.std(metricas_ran2[met])
  print(f'Média do {met}: {media}')
  print(f'Desvio {desvio}')
  print(f'Intervalo [{(media-(2*desvio)):.3f},{(media+(2*desvio)):.3f}]')
  print('-*-'*20)

-fit_time
-- [0.00136828 0.00124907 0.00824594 0.00139356 0.00130343 0.00134826
 0.00137329]
Média do fit_time: 0.0023259775979178293
Desvio 0.0024172434846046922
Intervalo [-0.003,0.007]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-score_time
-- [0.00338578 0.00456524 0.00357342 0.01053238 0.016397   0.00332665
 0.00372267]
Média do score_time: 0.006500448499407087
Desvio 0.004690429618825776
Intervalo [-0.003,0.016]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_accuracy
-- [0.78571429 0.67857143 0.64285714 0.7037037  0.62962963 0.55555556
 0.44444444]
Média do test_accuracy: 0.6343537414965986
Desvio 0.10144946365859885
Intervalo [0.431,0.837]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
-test_precision_macro
-- [0.81111111 0.70467836 0.79166667 0.75357143 0.79166667 0.76
 0.39583333]
Média do test_precision_macro: 0.7155039384174723
Desvio 0.13441820453314032
Intervalo [0.447,0.984]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*

Comparando , temos:

- naive bayes *sem otimização*:
  - acuracia: 0,722
  - precisão: 0,754
  - recall: 0,724
- naive bayes *com grid search*:
  - acuracia: 0,712
  - precisão: 0,748
  - recall: 0,713
- naive bayes com *random search*:
  - acuracia: 0,634
  - precisão: 0,715
  - recall: 0,635

Temos então que os resultados nã foram melhores